In [28]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import polars as pl
import pytomlpp as toml
import pandas as pd
from data_preprocessing import getXY, data_normalizeation
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from tqdm import tqdm

In [29]:
cfg = toml.load('/home/gyli/GP_HP/cfg/feature_selection.toml')
data_path = cfg['data_path']
variance_filter_thres = cfg['variance_filter_thres']
raw_x, raw_y = getXY(data_path)
# raw_x = data_normalizeation(raw_x)
future_names = raw_x.columns
X, y = raw_x.select(['年龄', '淋巴细胞数', '白蛋白', '活化部分凝血活酶时间', '低密度脂蛋白', '钠', '中性粒细胞数', '甘油三酯', '凝血酶原时间', '红细胞平均体积', '总胆红素']).to_pandas(), np.ravel(raw_y.to_pandas())

In [30]:
knn_model = KNeighborsClassifier(n_neighbors=3)

# 创建随机森林模型
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 创建GBDT模型
gbdt_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# 创建Pipeline来对数据进行归一化处理
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 归一化处理
    ('model', knn_model)  # 在这里更改模型
])

# 对K近邻模型进行交叉验证
knn_scores = cross_val_score(pipeline, X, y, cv=5)
print("K近邻模型交叉验证结果:", knn_scores,'均值',knn_scores.mean(),'方差',knn_scores.std())

# 创建随机森林模型
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 创建GBDT模型
gbdt_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# 对随机森林模型进行交叉验证
rf_scores = cross_val_score(rf_model, X, y, cv=5)
print("随机森林模型交叉验证结果:", rf_scores,'均值',rf_scores.mean(),'方差',rf_scores.std())

# 对GBDT模型进行交叉验证
gbdt_scores = cross_val_score(gbdt_model, X, y, cv=5)
print("GBDT模型交叉验证结果:", gbdt_scores,'均值',gbdt_scores.mean(),'方差',gbdt_scores.mean())

K近邻模型交叉验证结果: [0.75005031 0.60487118 0.80595813 0.77355072 0.63486312] 均值 0.7138586936267337 方差 0.07933959601975767
随机森林模型交叉验证结果: [0.88206883 0.70229469 0.91626409 0.90257649 0.7071256 ] 均值 0.8220659392581833 方差 0.09644897164433966
GBDT模型交叉验证结果: [0.74763534 0.54066023 0.87902576 0.85386473 0.61976651] 均值 0.7281905138752375 方差 0.7281905138752375


In [31]:
# knn_df = pd.DataFrame(columns=['n_neighbors', 'weights', 'p_val', 'cross_val_score', 'mean_score', 'std'])
knn_dfs = []
n_neighbors = [i for i in range(1, 20)]
weights = ['uniform', 'distance']
p_vals = [i for i in range(1,10)]
for k in n_neighbors:
    for w in weights:
        for p in p_vals:
            knn_model = KNeighborsClassifier(n_neighbors=k, weights=w,p=p)
            pipeline = Pipeline([
                ('scaler', StandardScaler()),  # 归一化处理
                ('model', knn_model)  # 在这里更改模型
            ])
            # 对K近邻模型进行交叉验证
            knn_scores = cross_val_score(pipeline, X, y, cv=5)
            row = {'n_neighbors' : k, 'weights' : w, 'p_val' : p, 'mean_score' : knn_scores.mean(), 'std' : knn_scores.std()}
            for i, acc in enumerate(knn_scores):
                row[f'cross_val_score_{i+1}'] = knn_scores[i]
            # knn_df = knn_df.add(row, ignore_index=True)
            knn_dfs.append(pd.DataFrame(row, index=[0]))
knn_df = pd.concat(knn_dfs)
knn_df.to_csv('/home/gyli/GP_HP/result/knn_results.csv')

In [32]:
knn_df

,n_neighbors,weights,p_val,mean_score,std,cross_val_score_1,cross_val_score_2,cross_val_score_3,cross_val_score_4,cross_val_score_5
0,1,uniform,1,0.790223,0.069729,0.849265,0.722826,0.857689,0.831320,0.690016
0,1,uniform,2,0.785674,0.071945,0.846448,0.713969,0.854267,0.829911,0.683776
0,1,uniform,3,0.785997,0.070294,0.843429,0.713969,0.856280,0.827899,0.688406
0,1,uniform,4,0.785071,0.069902,0.844033,0.714372,0.854469,0.825081,0.687399
0,1,uniform,5,0.785232,0.070526,0.844637,0.714372,0.855676,0.825081,0.686393
...,...,...,...,...,...,...,...,...,...,...
0,19,distance,5,0.790385,0.096057,0.829744,0.682367,0.908615,0.859702,0.671498
0,19,distance,6,0.788413,0.095221,0.828939,0.682166,0.904589,0.857085,0.669283
0,19,distance,7,0.787688,0.095603,0.826524,0.680757,0.904589,0.857890,0.668680
0,19,distance,8,0.787648,0.094858,0.826927,0.679952,0.904187,0.856079,0.671095


In [33]:
rf_dfs = []
# n_estimators = [i for i in range(100, 300, 100)]
# max_depth = [i for i in range(2, 3)]
# min_samples_leaf = [2]
# min_samples_split = [2]
# max_features = [None, 'sqrt', 'log2']
n_estimators = [i for i in range(5, 300, 5)]
max_depth = [i for i in range(2, 11)]
min_samples_leaf = [i for i in range(1, 5)]
min_samples_split = [i for i in range(2, 5)]
max_features = [None, 'sqrt', 'log2']

for n in n_estimators:
    for md in max_depth:
        for ml in  min_samples_leaf:
            for ms in min_samples_split:
                for mf in max_features:
                    rf_model = RandomForestClassifier(n_estimators=n, max_depth=md, min_samples_leaf=ml, min_samples_split=ms, max_features=mf, random_state=42)
                    rf_scores = cross_val_score(rf_model, X, y, cv=5)
                    row = {'n_estimators' : n, 'max_depth' : md, 'min_samples_leaf' : ml, 'min_samples_split' : ms, 'max_features' : mf, 'mean_score' : rf_scores.mean(), 'std' : rf_scores.std()}
                    for i, acc in enumerate(rf_scores):
                        row[f'cross_val_score_{i+1}'] = rf_scores[i]
                    rf_dfs.append(pd.DataFrame(row, index=[0]))
rf_df = pd.concat(rf_dfs)
rf_df.to_csv('/home/gyli/GP_HP/result/rf_results.csv')

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gyli/.local/lib/python3.10/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/home/gyli/.local/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/gyli/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 95, in validate_parameter_constraints
    raise InvalidParameterError(
sklearn.utils._param_validation.InvalidP

In [ ]:
rf_df

,n_estimators,max_depth,min_samples_leaf,min_samples_split,max_features,mean_score,std,cross_val_score_1,cross_val_score_2,cross_val_score_3,cross_val_score_4,cross_val_score_5
0,100,2,2,2,None,0.698160,0.142422,0.696518,0.486312,0.891103,0.806159,0.610709
0,100,2,2,2,sqrt,0.725857,0.126225,0.717046,0.569847,0.898752,0.834340,0.609300
0,100,2,2,2,log2,0.725857,0.126225,0.717046,0.569847,0.898752,0.834340,0.609300
0,200,2,2,2,None,0.698603,0.142235,0.697726,0.487118,0.891304,0.806159,0.610709
0,200,2,2,2,sqrt,0.725937,0.126904,0.716442,0.567834,0.901570,0.832729,0.611111
0,200,2,2,2,log2,0.725937,0.126904,0.716442,0.567834,0.901570,0.832729,0.611111


In [ ]:
gbdt_dfs = []
# n_estimators = [i for i in range(100, 300, 100)]
# learning_rate = [i * 0.05 for i in range(0, 20, 10)]
# subsample = [i*0.05+0.5 for i in range(0, 6, 3)]
n_estimators = [i for i in range(5, 300, 5)]
learning_rate = [round(i * 0.05, 3) for i in range(1, 20, 1)]
subsample = [round(i*0.05+0.4, 3) for i in range(0, 8, 1)]
loss = ['log_loss','exponential']
for n in n_estimators:
    for lr in learning_rate:
        for ss in subsample:
            for l in loss:
                gbdt_model = GradientBoostingClassifier(n_estimators=n, learning_rate=lr, subsample=ss, loss=l, random_state=42)
                gbdt_scores = cross_val_score(gbdt_model, X, y, cv=5)
                row = {'n_estimators' : n, 'learning_rate' : lr, 'subsample' : ss, 'loss' : l, 'mean_score' : gbdt_scores.mean(), 'std' : gbdt_scores.std()}
                for i, acc in enumerate(gbdt_scores):
                    row[f'cross_val_score_{i+1}'] = gbdt_scores[i]
                gbdt_dfs.append(pd.DataFrame(row, index=[0]))
gbdt_df = pd.concat(gbdt_dfs)
gbdt_df.to_csv('/home/gyli/GP_HP/result/gbdt_results.csv')

In [ ]:
gbdt_df

,n_estimators,learning_rate,subsample,loss,mean_score,std,cross_val_score_1,cross_val_score_2,cross_val_score_3,cross_val_score_4,cross_val_score_5
0,100,0.0,0.50,log_loss,0.545147,0.000080,0.545180,0.545089,0.545089,0.545089,0.545290
0,100,0.0,0.50,exponential,0.545147,0.000080,0.545180,0.545089,0.545089,0.545089,0.545290
0,100,0.0,0.65,log_loss,0.545147,0.000080,0.545180,0.545089,0.545089,0.545089,0.545290
0,100,0.0,0.65,exponential,0.545147,0.000080,0.545180,0.545089,0.545089,0.545089,0.545290
0,100,0.5,0.50,log_loss,0.724889,0.110215,0.748239,0.564815,0.853261,0.824074,0.634058
0,100,0.5,0.50,exponential,0.724688,0.112590,0.746830,0.559984,0.849034,0.834138,0.633454
0,100,0.5,0.65,log_loss,0.732337,0.111742,0.757094,0.567432,0.858897,0.835346,0.642915
0,100,0.5,0.65,exponential,0.723320,0.117529,0.743610,0.545290,0.856079,0.833333,0.638285
0,200,0.0,0.50,log_loss,0.545147,0.000080,0.545180,0.545089,0.545089,0.545089,0.545290
0,200,0.0,0.50,exponential,0.545147,0.000080,0.545180,0.545089,0.545089,0.545089,0.545290


In [ ]:
n_estimators = [i for i in range(5, 300, 5)]
learning_rate = [round(i * 0.05, 3) for i in range(1, 20, 1)]
subsample = [round(i*0.05+0.4, 3) for i in range(0, 8, 1)]

In [ ]:
subsample

[0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]